# Mediapipe

### la bibliothèque python mediapipe pour détecter les repères du visage et de la main. Nous utiliserons un modèle holistique des solutions mediapipe pour détecter tous les repères du visage et de la main. Nous verrons également comment accéder à différents repères du visage et des mains qui peuvent être utilisés pour différentes applications de vision par ordinateur telles que la détection du langage des signes, la détection de la somnolence, etc.

### Bibliothèques requises

   * Mediapipe est une bibliothèque multiplateforme développée par Google qui fournit d'étonnantes solutions ML prêtes à l'emploi pour les tâches de vision par ordinateur.
   * La bibliothèque OpenCV en python est une bibliothèque de vision par ordinateur largement utilisée pour l'analyse d'images, le traitement d'images, la détection, la reconnaissance, etc.
   
### Installation des bibliothèques requises

     pip installer opencv-python mediapipe
     
### Vous trouverez ci-dessous l'approche par étapes pour la détection des points de repère du visage et de la main

## ÉTAPE-1 : Importez toutes les bibliothèques nécessaires. Dans notre cas, seules deux bibliothèques sont nécessaires.

In [9]:
# Import Libraries
import cv2
import time
import mediapipe as mp

## ÉTAPE-2 : Initialisation du modèle holistique et des utilitaires de dessin pour détecter et dessiner des points de repère sur l'image.

In [10]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Examinons les paramètres du modèle holistique :
       Holistique(

      static_image_mode=Faux,

      model_complexity=1,

      smooth_landmarks=Vrai,

      min_detection_confidence=0.5,

      min_tracking_confidence=0.5

        )

    * static_image_mode : Il est utilisé pour spécifier si les images d'entrée doivent être traitées comme des images statiques ou comme un flux vidéo. La valeur par défaut est Faux.
    
    * model_complexity : Il est utilisé pour spécifier la complexité du modèle de point de repère de pose : 0, 1 ou 2. À mesure que la complexité du modèle du modèle augmente, la précision du point de repère et la latence augmentent. La valeur par défaut est 1.
    
    * smooth_landmarks : ce paramètre est utilisé pour réduire la gigue dans la prédiction en filtrant les repères de pose sur différentes images d'entrée. La valeur par défaut est Vrai.
    * min_detection_confidence : il est utilisé pour spécifier la valeur de confiance minimale avec laquelle la détection à partir du modèle de détection de personne doit être considérée comme réussie. Peut spécifier une valeur dans [0.0,1.0]. La valeur par défaut est 0,5.
    * min_tracking_confidence : il est utilisé pour spécifier la valeur de confiance minimale avec laquelle la détection à partir du modèle de suivi de points de repère doit être considérée comme réussie. Peut spécifier une valeur dans [0.0,1.0]. La valeur par défaut est 0,5.

## ÉTAPE-3 : détection des points de repère du visage et de la main à partir de l'image. Le modèle holistique traite l'image et produit des points de repère pour le visage, la main gauche, la main droite et détecte également la pose du 

    1. Capturez les images en continu à partir de la caméra à l'aide d'OpenCV.
    2. Convertissez l'image BGR en image RVB et faites des prédictions à l'aide d'un modèle holistique initialisé.
    3. Les prédictions faites par le modèle holistique sont enregistrées dans la variable de résultats à partir de laquelle nous pouvons accéder aux points de repère en utilisant respectivement results.face_landmarks, results.right_hand_landmarks, results.left_hand_landmarks.
    4.Dessinez les points de repère détectés sur l'image à l'aide de la fonction draw_landmarks des utilitaires de dessin.
    5.Affichez l'image résultante

In [11]:
# (0) in VideoCapture is used to connect to your computer's default camera
capture = cv2.VideoCapture(0)

# Initializing current time and precious time for calculating the FPS
previousTime = 0
currentTime = 0

while capture.isOpened():
    # capture frame by frame
    ret, frame = capture.read()

    # resizing the frame for better view
    frame = cv2.resize(frame, (800, 600))

    # Converting the from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Making predictions using holistic model
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True

    # Converting back the RGB image to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Drawing the Facial Landmarks
    mp_drawing.draw_landmarks(
    image,
    results.face_landmarks,
    mp_holistic.FACEMESH_CONTOURS,
    mp_drawing.DrawingSpec(
        color=(255,0,255),
        thickness=1,
        circle_radius=1
    ),
    mp_drawing.DrawingSpec(
        color=(0,255,255),
        thickness=1,
        circle_radius=1
    )
    )

    # Drawing Right hand Land Marks
    mp_drawing.draw_landmarks(
    image,
    results.right_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS
    )

    # Drawing Left hand Land Marks
    mp_drawing.draw_landmarks(
    image,
    results.left_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS
    )
    
    # Calculating the FPS
    currentTime = time.time()
    fps = 1 / (currentTime-previousTime)
    previousTime = currentTime

    # Displaying FPS on the image
    cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

    # Display the resulting image
    cv2.imshow("Facial and Hand Landmarks", image)

    # Enter key 'q' to break the loop
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# When all the process is done
# Release the capture and destroy all windows
capture.release()
cv2.destroyAllWindows()


### Le modèle holistique produit 468 repères de visage, 21 repères de gauche et 21 repères de droite. Les points de repère individuels sont accessibles en spécifiant l'index du point de repère requis. Exemple : résultats.left_hand_landmarks.landmark[0]. Vous pouvez obtenir l'index de tous les points de repère individuels en utilisant le code ci-dessous :

In [12]:
# Code to access landmarks
for landmark in mp_holistic.HandLandmark:
    print(landmark, landmark.value)
 
print(mp_holistic.HandLandmark.WRIST.value)

HandLandmark.WRIST 0
HandLandmark.THUMB_CMC 1
HandLandmark.THUMB_MCP 2
HandLandmark.THUMB_IP 3
HandLandmark.THUMB_TIP 4
HandLandmark.INDEX_FINGER_MCP 5
HandLandmark.INDEX_FINGER_PIP 6
HandLandmark.INDEX_FINGER_DIP 7
HandLandmark.INDEX_FINGER_TIP 8
HandLandmark.MIDDLE_FINGER_MCP 9
HandLandmark.MIDDLE_FINGER_PIP 10
HandLandmark.MIDDLE_FINGER_DIP 11
HandLandmark.MIDDLE_FINGER_TIP 12
HandLandmark.RING_FINGER_MCP 13
HandLandmark.RING_FINGER_PIP 14
HandLandmark.RING_FINGER_DIP 15
HandLandmark.RING_FINGER_TIP 16
HandLandmark.PINKY_MCP 17
HandLandmark.PINKY_PIP 18
HandLandmark.PINKY_DIP 19
HandLandmark.PINKY_TIP 20
0
